In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler
import os

# Note: In the sixdof android app "enable ctrl" need to uncheck first

### Record current joint as posture, and ask robot to move to certain posture

In [2]:
class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        rclpy.spin(self.node)

In [3]:
rclpy.init()
cwd=os.getcwd()
urdf_file = cwd+'/sawyer.urdf'
robot = URDFModel(urdf_file)
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}

node = Node('pub_dbg')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)
pass 

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [15]:
# robot.EENames

In [4]:
# ros_interface.spin_thread()
t1 = Thread(target=ros_interface.spin_thread)
t1.start()

In [8]:
# apos=0
# def goto_q_pos_velocity_ctrl(target_q, alpha=0.7, tol=0.01,  delay=0.1):
#     global apos
#     target_jointpos=target_q
#     cpos=robot.getJoints()
#     dpos=np.linalg.norm(target_jointpos-cpos)

#     while dpos>tol: 
#         cpos=robot.getJoints()
#         dpos=np.linalg.norm(target_jointpos-cpos)
          

#         qd =(target_jointpos-cpos)*alpha
#         apos=qd 
#         if np.max(qd) <0.001:
#             print('reached')
#             break 


#         cmd_msg = JointCommand()
#         cmd_msg.names = robot.jointNames
#         cmd_msg.mode = cmd_msg.VELOCITY_MODE
#         cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
#         cmd_msg.velocity = list(qd)
#         cmd_pub.publish(cmd_msg)
#         time.sleep(delay)

In [5]:
def set_vels(vels):
    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity =vels 
    cmd_pub.publish(cmd_msg)

In [6]:
apos=0
def goto_q_pos_velocity_ctrl(target_q, alpha=0.7, tol=0.01,  delay=0.1, maxstep=50):
    global apos
    target_jointpos=target_q
    cpos=robot.getJoints()
    dpos=np.linalg.norm(target_jointpos-cpos)

    step =0
    while dpos>tol: 
        cpos=robot.getJoints()
        dpos=np.linalg.norm(target_jointpos-cpos)
          

        qd =(target_jointpos-cpos)*alpha
        apos=qd 
        if np.max(qd) <0.001:
            print('reached')
            break 


        set_vels(list(qd))
        time.sleep(delay)
        step +=1

        if step>=maxstep:
            break 
        
    set_vels([0.0]*10)

## Step 1: Safety test

<b> 1.1 record current posture </b>

In [7]:
pos1=robot.getJoints()
pos1

array([ 0.62475781, -0.66947949,  0.49860254, -2.40586523,  1.13731641,
       -1.59123926, -1.77138672,  2.88361816,  0.        ,  0.        ])

<b> 1.2 Move the robot few centimeters manually </b>

<b> 1.3 goto to recorded pos1 </b>

In [9]:
goto_q_pos_velocity_ctrl(pos1, tol=0.05)

<b> Test done </b>

## Step 2: Experiment

### read current posture to save for different task

In [8]:
posture=robot.getJoints()
posture

array([ 0.57437793, -0.67592773,  0.6533125 , -2.36299512,  1.27580859,
       -1.42587598, -1.64586621,  2.84970117,  0.        ,  0.        ])

### Task: Block lifting 

In [8]:
#posture saved Nov 8, 23
block_start_pos = np.array([ 0.25362402, -0.1616416 ,  0.2595332 , -1.92522363,  1.4523877 ,
       -1.24452832, -1.8799541 ,  0.57152539,  0.03735149,  0.        ])

In [ ]:
goto_q_pos_velocity_ctrl(block_start_pos, tol=0.11)

### Drawer close

In [10]:
ee_init_pos_dclose=np.array([ 0.57437793, -0.67592773,  0.6533125 , -2.36299512,  1.27580859,
       -1.42587598, -1.64586621,  2.84970117,  0.        ,  0.        ])

In [11]:
goto_q_pos_velocity_ctrl(ee_init_pos_dclose, tol=0.05)